In [6]:
%pip install chromadb faiss-cpu lancedb

     ---------------------------------------- 0.0/67.3 kB ? eta -:--:--
     ---------------------------------------- 0.0/67.3 kB ? eta -:--:--
     ------ --------------------------------- 10.2/67.3 kB ? eta -:--:--
     ----------------- -------------------- 30.7/67.3 kB 435.7 kB/s eta 0:00:01
     ----------------------- -------------- 41.0/67.3 kB 326.8 kB/s eta 0:00:01
     ---------------------------------- --- 61.4/67.3 kB 365.7 kB/s eta 0:00:01
     -------------------------------------- 67.3/67.3 kB 331.5 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ------------------------

In [7]:
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
raw_documents = TextLoader('example_data/state_of_the_union.txt').load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)

## Chroma

In [20]:

from langchain_community.vectorstores import Chroma

db = Chroma.from_documents(documents, OpenAIEmbeddings())

## FAISS

In [23]:
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(documents, OpenAIEmbeddings())

## Lance

In [11]:
from langchain_community.vectorstores import LanceDB

import lancedb

db = lancedb.connect("tmp/lancedb")
table = db.create_table(
    "my_table",
    data=[
        {
            "vector": OpenAIEmbeddings().embed_query("Hello World"),
            "text": "Hello World",
            "id": "1",
        }
    ],
    mode="overwrite",
)

db = LanceDB.from_documents(documents, OpenAIEmbeddings())

# Similarity search

In [24]:
query = "What did the president say about Ketanji Brown Jackson"

docs = db.similarity_search(query)
print(docs[0].page_content)

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nationâ€™s top legal minds, who will continue Justice Breyerâ€™s legacy of excellence. 

A former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since sheâ€™s been nominated, sheâ€™s received a broad range of supportâ€”from the Fraternal Order of Police to former judges appointed by Democrats and Republicans. 

And if we are to advance liberty and justice, we need to secure the Border and fix the immigration system. 

We can do both. At our border, weâ€™ve installed new technology like cutting-edge scanners to better detect drug smuggling.  

Weâ€™ve set up joint patrols with Mexico and Guatemala to catch more human traffickers.


In [25]:
embedding_vector = OpenAIEmbeddings().embed_query(query)

docs = db.similarity_search_by_vector(embedding_vector)
print(docs[0].page_content)

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nationâ€™s top legal minds, who will continue Justice Breyerâ€™s legacy of excellence. 

A former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since sheâ€™s been nominated, sheâ€™s received a broad range of supportâ€”from the Fraternal Order of Police to former judges appointed by Democrats and Republicans. 

And if we are to advance liberty and justice, we need to secure the Border and fix the immigration system. 

We can do both. At our border, weâ€™ve installed new technology like cutting-edge scanners to better detect drug smuggling.  

Weâ€™ve set up joint patrols with Mexico and Guatemala to catch more human traffickers.


# Asynchronous operations

In [26]:
%pip install qdrant-client

     ---------------------------------------- 0.0/57.5 kB ? eta -:--:--
     ---------------------------------------- 57.5/57.5 kB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/206.3 kB ? eta -:--:--
   ------- -------------------------------- 41.0/206.3 kB 2.0 MB/s eta 0:00:01
   ------------- -------------------------- 71.7/206.3 kB 1.3 MB/s eta 0:00:01
   --------------------- ------------------ 112.6/206.3 kB 1.1 MB/s eta 0:00:01
   --------------------- ------------------ 112.6/206.3 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 206.3/206.3 kB 1.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.1 MB 991.0 kB/s eta 0:00:02
   --- ------------------------------------ 0.1/1.1 MB 1.1 MB/s eta 0:00:01
   ----- ---------------------------------- 0.1/1.1 MB 1.2 MB/s eta 0:00:01
   ----- ---------------------------------- 0.2/1.1 MB 1.1 MB/s eta 0:00:01
   -

In [36]:
from langchain_community.vectorstores import Qdrant

db = await Qdrant.afrom_documents(documents, OpenAIEmbeddings(), "http://localhost:6333")


TypeError: VectorStore.afrom_documents() takes 3 positional arguments but 4 were given

In [30]:
query = "What did the president say about Ketanji Brown Jackson"

docs = await db.asimilarity_search(query)
print(docs[0].page_content)

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nationâ€™s top legal minds, who will continue Justice Breyerâ€™s legacy of excellence. 

A former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since sheâ€™s been nominated, sheâ€™s received a broad range of supportâ€”from the Fraternal Order of Police to former judges appointed by Democrats and Republicans. 

And if we are to advance liberty and justice, we need to secure the Border and fix the immigration system. 

We can do both. At our border, weâ€™ve installed new technology like cutting-edge scanners to better detect drug smuggling.  

Weâ€™ve set up joint patrols with Mexico and Guatemala to catch more human traffickers.


In [35]:
embedding_vector = OpenAIEmbeddings().embed_query(query)
docs = await db.asimilarity_search_by_vector(embedding_vector)
print(docs[0].page_content)

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nationâ€™s top legal minds, who will continue Justice Breyerâ€™s legacy of excellence. 

A former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since sheâ€™s been nominated, sheâ€™s received a broad range of supportâ€”from the Fraternal Order of Police to former judges appointed by Democrats and Republicans. 

And if we are to advance liberty and justice, we need to secure the Border and fix the immigration system. 

We can do both. At our border, weâ€™ve installed new technology like cutting-edge scanners to better detect drug smuggling.  

Weâ€™ve set up joint patrols with Mexico and Guatemala to catch more human traffickers.


# Maximum marginal relevance search (MMR)

In [39]:
query = "What did the president say about Ketanji Brown Jackson"

found_docs = db.max_marginal_relevance_search(query, k=2, fetch_k=10)
for i, doc in enumerate(found_docs):
    print(f"{i + 1}.", doc.page_content, "\n")

1. One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nationâ€™s top legal minds, who will continue Justice Breyerâ€™s legacy of excellence. 

A former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since sheâ€™s been nominated, sheâ€™s received a broad range of supportâ€”from the Fraternal Order of Police to former judges appointed by Democrats and Republicans. 

And if we are to advance liberty and justice, we need to secure the Border and fix the immigration system. 

We can do both. At our border, weâ€™ve installed new technology like cutting-edge scanners to better detect drug smuggling.  

Weâ€™ve set up joint patrols with Mexico and Guatemala to catch more human traffickers. 


C:\Users\linkiewicz_k\AppData\Local\Temp\ipykernel_30352\4040875783.py:3: RuntimeWarning: coroutine 'FAISS.amax_marginal_relevance_search' was never awaited
  found_docs = db.max_marginal_relevance_search(query, k=2, fetch_k=10)
